In [1]:
from datetime import datetime, timedelta
import pickle
from time import sleep

from bs4 import BeautifulSoup
from requests_html import HTMLSession
import yfinance as yf

In [2]:
EXCHANGE_PATHS = {
    'nasdaq': 'nasdaq',
    'nyse': 'newyorkstockexchange',
    'amex': 'americanstockexchange'}
base_url = 'https://www.advfn.com'
session = HTMLSession()

In [3]:
def get_soup(url):
    try:
        res = session.get(url)
        html = res.html.html
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    except BaseException as e:
        print(f'Problem retrieving page: {url}\n{e}')

In [4]:
def get_table(soup):
    try:
        table = soup.find('table', class_='market')
        return table
    except BaseExcetpion as e:
        print('Unable to obtain table')
        print(e)

In [5]:
def get_rows(table):
    try:
        rows = table.find_all('tr')
        rows = [
            row for row in rows if 'class' in row.attrs 
            and row.attrs['class'][0].startswith('ts')]
        return rows
    except BaseException as e:
        print('Unable to extract rows')
        print(e)

In [6]:
def is_valid(symbol):
    MAXLEN = 4
    return len(symbol) <= MAXLEN and symbol.isalpha()

In [7]:
def get_symbol(row):
    SYMBOL_COL_IDX = 1
    try:
        symbol = row.find_all('a')[SYMBOL_COL_IDX].text
        if is_valid(symbol):
            return symbol
    except BaseException as e:
        print('Unable to get symbol from row:', row)
        print(e)

In [8]:
def extract_symbols(soup):
    symbols = []
    table = get_table(soup)
    rows = get_rows(table)
    for row in rows:
        symbol = get_symbol(row)
        if symbol is not None:
            symbols.append(symbol)
    return symbols

In [9]:
def extract_all_symbols():
    all_symbols = []
    for exchange, path in EXCHANGE_PATHS.items():
        print(f'Beggining {exchange}\nLetter:')
        for letter in list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
            print(letter, end='')
            url = f'{base_url}/{exchange}/{path}.asp?companies={letter}'
            soup = get_soup(url)
            symbols = extract_symbols(soup)
            all_symbols += symbols
        print()
    return sorted(all_symbols)

In [10]:
all_symbols = extract_all_symbols()

Beggining nasdaq
Letter:
ABCDEFGHIJKLMNOPQRSTUVWXYZ
Beggining nyse
Letter:
ABCDEFGHIJKLMNOPQRSTUVWXYZ
Beggining amex
Letter:
ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [11]:
all_symbols = sorted(list(set(all_symbols)))

In [12]:
all_symbols[:10]

['A', 'AA', 'AAAP', 'AABA', 'AABC', 'AAC', 'AACC', 'AACE', 'AACI', 'AACQ']

In [13]:
all_symbols[-10:]

['ZWRK', 'ZWS', 'ZY', 'ZYME', 'ZYNE', 'ZYXI', 'ZZK', 'ZZO', 'ZZX', 'ZZZ']

In [14]:
with open('../data/all_symbols.pkl', 'wb') as f:
    pickle.dump(all_symbols, f)

In [15]:
n = len(all_symbols)
n

11767

In [16]:
END = datetime.now()
START = END - timedelta(7)
test = yf.download(['AAPL', 'TSLA', 'ZQZZ'], start=str(START.date()), end=str(END.date()))['Adj Close']
test

[*********************100%%**********************]  3 of 3 completed

1 Failed download:
['ZQZZ']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Ticker,AAPL,TSLA,ZQZZ
Date,,,
2024-09-23,226.470001,250.000000,NaN
2024-09-24,227.369995,254.270004,NaN
2024-09-25,226.369995,257.019989,NaN
2024-09-26,227.520004,254.220001,NaN
2024-09-27,227.789993,260.459991,NaN


In [17]:
test2 = test.loc[:, test.iloc[-1, :].notnull()]
test2

Ticker,AAPL,TSLA
Date,,
2024-09-23,226.470001,250.000000
2024-09-24,227.369995,254.270004
2024-09-25,226.369995,257.019989
2024-09-26,227.520004,254.220001
2024-09-27,227.789993,260.459991


In [18]:
test2.columns

Index(['AAPL', 'TSLA'], dtype='object', name='Ticker')

In [19]:
start = str(START.date())
end = str(END.date())
start, end

('2024-09-22', '2024-09-29')

### Next Time
Code below is inefficient, bc it retries symbols if < $2 instead of removing them. 
Add to `omit` instead and do not rerun. (Update for all code chunks below.)

In [20]:
i = 0
active = []
failed = []
omit = []

while i <= n:
    first = i
    last = min(i + 100, n)
    batch = all_symbols[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    df = df.loc[:, df.iloc[-1, :].notnull()]
    batch_active = df.columns
    # Exclude if current val < 2.00
    df = df.loc[:, df.iloc[-1, :] >= 2.]
    batch_omit = [sym for sym in batch_active if sym not in df.columns]
    omit += list(batch_omit)
    active += list(df.columns)
    fails = [sym for sym in batch if sym not in df.columns and sym not in batch_omit]
    failed += fails
    sleep(3)
    i += 100

[****                   9%%                      ]  9 of 100 completed

$AAAP: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$AAII: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$AATI: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ABBK: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$AACI: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)


[*******               15%%                      ]  15 of 100 completed

$ABGX: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)


[***********           23%%                      ]  23 of 100 completed

...

[**********************84%%**************        ]  56 of 67 completed

$ZNDT: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ZLTQ: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)


[**********************97%%********************* ]  65 of 67 completed

$ZRBA: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)


[*********************100%%**********************]  67 of 67 completed

38 Failed downloads:
['ZMBA', 'ZONA', 'ZOLL', 'ZOLT', 'ZOES', 'ZVXI', 'ZONS', 'ZILG', 'ZLRG', 'ZIPR', 'ZPIN', 'ZOMX', 'ZKID', 'ZOO', 'ZRAN', 'ZNDT', 'ZLTQ', 'ZRBA']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)')
['ZNGA', 'ZT', 'ZY', 'ZWRK', 'ZVO', 'ZIXI', 'ZING', 'ZSAN', 'ZNH', 'ZIOP', 'ZME', 'ZYNE', 'ZN']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['ZZK', 'ZTST', 'ZZO', 'ZZX', 'ZNTE', 'ZOOG', 'ZVV']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


In [21]:
'ACAI' in active, 'ACAI' in failed, 'ACAI' in omit 

(False, False, True)

### 2nd Attempt

In [22]:
len(active), len(omit)

(4790, 1067)

In [23]:
n = len(failed)
n

5910

In [24]:
failed_again = []
i = 0
while i <= n:
    first = i
    last = min(i + 100, n)
    batch = failed[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    print(df.shape)
    if len(df):
        df = df.loc[:, df.iloc[-1, :].notnull()]
        batch_active = df.columns
        # Exclude if current val < 2.00
        df = df.loc[:, df.iloc[-1, :] >= 2.]
        batch_omit = [sym for sym in batch_active if sym not in df.columns]
        omit += list(batch_omit)
        active += list(df.columns)
        fails = [sym for sym in batch if sym not in df.columns and sym not in batch_omit]
        failed_again += fails
    else:
        failed_again += batch
    sleep(3)
    i += 100

[***                    6%%                      ]  6 of 100 completed

$AATI: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$AACI: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ACCL: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ACCI: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ACEC: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ABER: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$AAAP: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)


[******                12%%                      ]  12 of 100 completed

$AAII: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ABBK: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ADBL: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ACW: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)


[*******               15%%                      ]  15 of 100 completed

...

$ZENX: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)$YAVY: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)

(0, 100)


[**********************50%%                      ]  5 of 10 completed

$ZVXI: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)


[*********************100%%**********************]  10 of 10 completed

10 Failed downloads:
['ZZK', 'ZZX', 'ZTST', 'ZZO', 'ZVV']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
['ZVXI']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)')
['ZY', 'ZWRK', 'ZYNE', 'ZVO']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


(0, 10)


### 3rd Attempt

In [25]:
'ACAI' in active, 'ACAI' in failed, 'ACAI' in omit 

(False, False, True)

In [26]:
len(active), len(omit)

(4797, 1068)

In [27]:
failed = failed_again[:]
n = len(failed)
n

5902

In [28]:
failed_again = []
i = 0
while i <= n :
    first = i
    last = min(i + 100, n)
    batch = failed[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    print(df.shape)
    if len(df):
        df = df.loc[:, df.iloc[-1, :].notnull()]
        batch_active = df.columns
        # Exclude if current val < 2.00
        df = df.loc[:, df.iloc[-1, :] >= 2.]
        batch_omit = [sym for sym in batch_active if sym not in df.columns]
        omit += list(batch_omit)
        active += list(df.columns)
        fails = [sym for sym in batch if sym not in df.columns and sym not in batch_omit]
        failed_again += fails
    else:
        failed_again += batch
    sleep(3)
    i += 100

[***                    6%%                      ]  6 of 100 completed

$ACCL: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$AATI: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ACCI: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ACEC: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$AACI: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ABER: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$AAAP: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)


[*****                 11%%                      ]  11 of 100 completed

$AAII: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ADBL: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ACW: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ABBK: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)


[*******               15%%                      ]  15 of 100 completed

...

$YAVY: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)
$ZENX: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)


[*********************100%%**********************]  100 of 100 completed

100 Failed downloads:
['ZMBA', 'ZFC', 'YZC', 'YAKC', 'ZONS', 'XTND', 'ZOMX', 'ZKID', 'ZNDT', 'YCB', 'YDIW', 'ZOLL', 'XTXI', 'XYNY', 'YESR', 'ZOES', 'XTLY', 'ZIGO', 'YDIV', 'YOKU', 'YANB', 'YTEC', 'ZEPE', 'ZILA', 'ZRBA', 'XVIX', 'ZONA', 'ZICA', 'ZVXI', 'XXIA', 'ZOO', 'ZRAN', 'ZLTQ', 'XXV', 'ZOLT', 'YGRO', 'YDNT', 'ZILG', 'ZAIS', 'YDKN', 'ZIPR', 'YTHK', 'ZPIN', 'ZLRG', 'XUE', 'YAVY', 'ZENX']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-09-22 -> 2024-09-29)')
['ZNGA', 'ZGYH', 'ZARS', 'ZAPZ', 'ZAGG', 'YGTY', 'ZIXI', 'ZHIC', 'YGYI', 'XSPA', 'YOGA', 'ZT', 'YMTX', 'YHOO', 'YRCW', 'ZAYO', 'ZNH', 'ZDIV', 'YELL', 'ZN', 'ZEV', 'YUMA', 'YAC', 'ZEN', 'YTPG', 'ZYNE', 'YPRO', 'ZFGN', 'ZIOP', 'ZME', 'ZGNX', 'YGF', 'ZBZX', 'ZVO', 'ZY', 'XSPL', 'ZSAN', 'ZING', 'YSAC', 'ZEST', 'ZWRK']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['ZNTE', 'ZOOG', 'ZTST', 'YZCM', 'ZZK', 'Y

(0, 100)


[*********************100%%**********************]  2 of 2 completed

2 Failed downloads:
['ZZX', 'ZZO']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


(0, 2)


### 4th Attempt

In [29]:
len(active), len(omit)

(4797, 1068)

In [30]:
failed = failed_again[:]
n = len(failed)
n

5902

In [31]:
with open('../data/all_symbols.pkl', 'wb') as f:
    pickle.dump(active, f)

In [32]:
'INTI' in active, 'INTI' in omit

(False, True)

In [33]:
'TSLA' in active, 'TSLA' in omit

(True, False)

In [ ]:
!say complete

In [ ]:
# Run...?
'''
failed_again = []
i = 0
while i <= n - 100:
    first = i
    last = min(i + 100, n)
    batch = failed[first:last]
    print(f'Batch: {batch[0]} - {batch[-1]}', end='\r')
    df = yf.download(batch, start=start, end=end)['Adj Close']
    print(df.shape)
    if len(df):
        df = df.loc[:, df.iloc[-1, :].notnull()]
        batch_active = df.columns
        # Exclude if current val < 2.00
        df = df.loc[:, df.iloc[-1, :] >= 2.]
        batch_omit = [sym for sym in batch_active if sym not in df.columns]
        omit += list(batch_omit)
        active += list(df.columns)
        fails = [sym for sym in batch if sym not in df.columns and sym not in batch_omit]
        failed_again += fails
    else:
        failed_again += batch
    sleep(3)
    i += 100
'''